In [172]:
import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression
from sklearn import metrics 
from sklearn.neural_network import MLPClassifier

import matplotlib.pyplot as plt
import seaborn as sns

from xgboost import XGBClassifier

Simple logistic regression

In [173]:
data = pd.read_csv('data/ML_DATA_MXF.csv')
data.dropna(inplace= True)

true_label = data.BINARY_PHENOTYPE

features = ['d_volume', 'd_hydropathy_KD', 'd_hydropathy_WW',
       'd_MW', 'd_Pi', 'secondary_structure', 'phi', 'psi', 'residue_sasa', 
       'n_hbond_acceptors', 'n_hbond_donors', 'SASA', 'snap2_score', 'dist_MGB', 'dist_MGE', 'dist_MFX'] # 'B', 'C', 'E', 'G', 'H', 'T',

features = data[features]
features['secondary_structure_codes'] = pd.Categorical(features.secondary_structure, categories=features.secondary_structure.unique()).codes
features.drop('secondary_structure', inplace = True, axis = 1)

true_label
features

/var/folders/nq/gpbdk7y11kvd1d9nk9s77b4c0000gn/T/ipykernel_10622/3423287689.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['secondary_structure_codes'] = pd.Categorical(features.secondary_structure, categories=features.secondary_structure.unique()).codes
/var/folders/nq/gpbdk7y11kvd1d9nk9s77b4c0000gn/T/ipykernel_10622/3423287689.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features.drop('secondary_structure', inplace = True, axis = 1)


d_volume  d_hydropathy_KD  d_hydropathy_WW  d_MW  d_Pi    phi    psi  \
0        -51.0              3.1            -1.27 -58.0  3.20 -58.04 -41.61   
1         51.4              2.4            -0.86  28.0 -0.04 -61.83 -59.76   
2          3.0              0.0            -1.98  -1.0  2.64 -58.04 -41.61   
3        -51.0              3.1            -1.27 -58.0  3.20 -58.04 -41.61   
4          3.0              0.0            -1.98  -1.0  2.64 -58.04 -41.61   
...        ...              ...              ...   ...   ...    ...    ...   
1664      23.7             -0.8            -0.64  10.0  0.62 -67.15 -27.15   
1665      51.4              2.4            -0.86  28.0 -0.04 -61.83 -59.76   
1666      51.4              2.4            -0.86  28.0 -0.04 -61.83 -59.76   
1667     113.3             -4.1            -0.14  99.1  4.79 -61.79 -46.23   
1668      51.4              2.4            -0.86  28.0 -0.04 -61.83 -59.76   

      residue_sasa  n_hbond_acceptors  n_hbond_donors       SASA  snap2_score  \
0             97.4                1.0             1.0  88.155089           63   
1             47.0                2.0             0.0  55.555816          -64   
2             97.4                1.0             1.0  88.155089           56   
3             97.4                1.0             1.0  88.155089           63   
4             97.4                1.0             1.0  88.155089           56   
...            ...                ...             ...        ...          ...   
1664          64.8                1.0             0.0  65.228155           63   
1665          47.0                2.0             0.0  55.555816          -64   
1666          47.0                2.0             0.0  55.555816          -64   
1667           2.4                3.0             1.0   2.551348           33   
1668          47.0                2.0             0.0  55.555816          -64   

       dist_MGB   dist_MGE   dist_MFX  secondary_structure_codes  
0     18.494689   7.838828  14.503318                          0  
1     14.434102   5.483332  10.609526                          0  
2     18.494689   7.838828  14.503318                          0  
3     18.494689   7.838828  14.503318                          0  
4     18.494689   7.838828  14.503318                          0  
...         ...        ...        ...                        ...  
1664  13.712391   5.381126  11.010229                          0  
1665  14.434102   5.483332  10.609526                          0  
1666  14.434102   5.483332  10.609526                          0  
1667  29.300727  43.081517  41.932227                          0  
1668  14.434102   5.483332  10.609526                          0  

[1648 rows x 16 columns]

In [174]:
random_seed = 144

X_train,X_test,y_train,y_test=train_test_split(features,true_label,test_size=0.20,random_state = random_seed) 

In [175]:
random_seed = 144

folds = StratifiedKFold(n_splits = 10, random_state = random_seed, shuffle = True)

param_grid = [ 
                {   'penalty': ['l2'],
                    'solver'  : ['newton-cg', 'lbfgs', 'liblinear', 'newton-cholesky', 'sag', 'saga'], 
                    'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
                    'class_weight' : ['balanced', None]
                } 
            ]

gsvc = GridSearchCV(LogisticRegression(random_state=random_seed), param_grid, cv = folds, scoring = 'f1') #, scoring='f1' , scoring = 'recall'

In [176]:
gsvc.fit(X_train,y_train)
gsvc.cv_results_

/Users/viktoriabrunner/Documents/Studium/PhD/DPhil/predict_FQ_resistance/predict_FQ_resistance/.venv_fq/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:824: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/viktoriabrunner/Documents/Studium/PhD/DPhil/predict_FQ_resistance/predict_FQ_resistance/.venv_fq/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 813, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/viktoriabrunner/Documents/Studium/PhD/DPhil/predict_FQ_resistance/predict_FQ_resistance/.venv_fq/lib/python3.10/site-packages/sklearn/metrics/_scorer.py", line 266, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
  File "/Users/viktoriabrunner/Documents/Studium/PhD/DPhil/predict_FQ_resistance/predict_FQ_resistance/.venv_fq/lib/python3.10/site-packages/sk

{'mean_fit_time': array([0.0693012 , 0.02497535, 0.0093127 , 0.00965703, 0.02922795,
        0.02631192, 0.05062964, 0.02222145, 0.00723252, 0.0071754 ,
        0.03376393, 0.0296833 , 0.04970937, 0.02215331, 0.00829015,
        0.008884  , 0.02736852, 0.03265464, 0.07152083, 0.04999266,
        0.01516747, 0.00979419, 0.03638344, 0.03674526, 0.07594192,
        0.03854871, 0.02410855, 0.01371934, 0.03169105, 0.03199546,
        0.05496483, 0.02560201, 0.00925558, 0.00818706, 0.02874205,
        0.03072171, 0.08179131, 0.0260319 , 0.01419234, 0.01047976,
        0.03100433, 0.03169007, 0.07087739, 0.0425745 , 0.01635642,
        0.01081781, 0.03557119, 0.03971887, 0.09521182, 0.02658405,
        0.01757085, 0.01046567, 0.03157198, 0.03356657, 0.09265792,
        0.03247504, 0.01804671, 0.03789918, 0.03304546, 0.03609152,
        0.09238498, 0.03984563, 0.02396636, 0.01305718, 0.03066568,
        0.03273628, 0.09837866, 0.02674165, 0.01880879, 0.00882933,
        0.03164148, 0.0338129 ,

In [179]:
df = pd.DataFrame(gsvc.cv_results_)
df.columns

Index(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time',
       'param_C', 'param_class_weight', 'param_penalty', 'param_solver',
       'params', 'split0_test_score', 'split1_test_score', 'split2_test_score',
       'split3_test_score', 'split4_test_score', 'split5_test_score',
       'split6_test_score', 'split7_test_score', 'split8_test_score',
       'split9_test_score', 'mean_test_score', 'std_test_score',
       'rank_test_score'],
      dtype='object')

In [180]:
gsvc.best_params_

{'C': 0.001,
 'class_weight': 'balanced',
 'penalty': 'l2',
 'solver': 'newton-cg'}

In [181]:
gsvc.best_estimator_

LogisticRegression(C=0.001, class_weight='balanced', random_state=144,
                   solver='newton-cg')

In [182]:
logreg =  LogisticRegression(solver='newton-cg', random_state=random_seed, class_weight = 'balanced', C = 0.001, penalty = 'l2') 

In [183]:
logreg.fit(X_train,y_train) 

LogisticRegression(C=0.001, class_weight='balanced', random_state=144,
                   solver='newton-cg')

In [184]:
y_pred=logreg.predict(X_test) 

In [185]:
y_df = pd.DataFrame(y_pred)
y_df.value_counts()

R    281
S     49
Name: count, dtype: int64

In [186]:
cnf_matrix = metrics.confusion_matrix(y_test, y_pred) 

cnf_matrix 

array([[280,   4],
       [  1,  45]])

In [187]:
class_names=[0,1] # name  of classes 

fig, ax = plt.subplots() 

tick_marks = np.arange(len(class_names)) 

plt.xticks(tick_marks, class_names) 

plt.yticks(tick_marks, class_names) 

# create heatmap 

sns.heatmap(pd.DataFrame(cnf_matrix), annot=True, cmap="YlGnBu" ,fmt='g') 

ax.xaxis.set_label_position("top") 

plt.tight_layout() 

plt.title('Confusion matrix', y=1.1) 

plt.ylabel('Actual label') 

plt.xlabel('Predicted label')

Text(0.5, 427.9555555555555, 'Predicted label')

In [188]:
sensitivity = cnf_matrix[1,1]/(cnf_matrix[1,1]+cnf_matrix[1,0])
specificity = cnf_matrix[0,0]/(cnf_matrix[0,0]+cnf_matrix[0,1])
print(sensitivity, specificity)

0.9782608695652174 0.9859154929577465
